# Loading Required Libraries

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_columns', None)

from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler

# Loading Data

In [8]:
train = pd.read_csv("train_fNxu4vz.csv")
test  = pd.read_csv("test_fjtUOL8.csv")
sub = pd.read_csv("sample_submission_HSqiq1Q.csv")

# Basic Understandidng of Data

In [9]:
train.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
0,10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
1,10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
2,10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
3,10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
4,10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [10]:
test.head()

,Loan_ID,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
0,10164310,"27,500",10+ years,Mortgage,129000.0,VERIFIED - income,debt_consolidation,12.87,0,68.0,10,37,Male
1,10164311,"26,000",10+ years,NaN,110000.0,not verified,credit_card,11.37,0,NaN,6,23,Male
2,10164312,"6,075",< 1 year,Rent,75000.0,VERIFIED - income,debt_consolidation,6.83,2,NaN,5,20,Male
3,10164313,"12,000",10+ years,Mortgage,73000.0,VERIFIED - income source,debt_consolidation,7.76,0,NaN,6,8,Male
4,10164314,"35,000",< 1 year,Mortgage,156000.0,not verified,debt_consolidation,9.62,0,26.0,9,21,Male


In [11]:
train.describe()

,Loan_ID,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Interest_Rate
count,1.643090e+05,1.392070e+05,164309.000000,164309.000000,75930.000000,164309.000000,164309.000000,164309.000000
mean,1.008216e+07,7.333116e+04,17.207189,0.781698,34.229356,11.193818,25.067665,2.158951
std,4.743207e+04,6.037750e+04,7.845083,1.034747,21.761180,4.991813,11.583067,0.738364
min,1.000000e+07,4.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
25%,1.004108e+07,4.500000e+04,11.370000,0.000000,16.000000,8.000000,17.000000,2.000000
50%,1.008216e+07,6.300000e+04,16.840000,0.000000,31.000000,10.000000,23.000000,2.000000
75%,1.012323e+07,8.869750e+04,22.780000,1.000000,50.000000,14.000000,32.000000,3.000000
max,1.016431e+07,7.500000e+06,39.990000,8.000000,180.000000,76.000000,156.000000,3.000000


In [12]:
test.describe()

,Loan_ID,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts
count,1.095410e+05,9.264300e+04,109541.000000,109541.000000,50682.000000,109541.000000,109541.000000
mean,1.021908e+07,7.348541e+04,17.228969,0.788810,33.914684,11.174337,25.068440
std,3.162191e+04,5.563845e+04,7.847310,1.039903,21.732856,4.946314,11.599639
min,1.016431e+07,3.000000e+03,0.000000,0.000000,0.000000,0.000000,2.000000
25%,1.019170e+07,4.500000e+04,11.350000,0.000000,15.000000,8.000000,17.000000
50%,1.021908e+07,6.300000e+04,16.860000,0.000000,31.000000,10.000000,24.000000
75%,1.024646e+07,8.900000e+04,22.780000,1.000000,49.000000,14.000000,32.000000
max,1.027385e+07,7.446395e+06,39.990000,8.000000,149.000000,84.000000,124.000000


In [13]:
train.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

In [14]:
test.isnull().sum()

Loan_ID                        0
Loan_Amount_Requested          0
Length_Employed             4936
Home_Owner                 16711
Annual_Income              16898
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    58859
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
dtype: int64

# Concating train and test data into single Dataframe for preprocessing

In [15]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df.dtypes

Annual_Income              float64
Debt_To_Income             float64
Gender                      object
Home_Owner                  object
Income_Verified             object
Inquiries_Last_6Mo           int64
Interest_Rate              float64
Length_Employed             object
Loan_Amount_Requested       object
Loan_ID                      int64
Months_Since_Deliquency    float64
Number_Open_Accounts         int64
Purpose_Of_Loan             object
Total_Accounts               int64
train_or_test               object
dtype: object

In [17]:
df['Purpose_Of_Loan'].value_counts()

debt_consolidation    161403
credit_card            61394
home_improvement       15455
other                  13846
major_purchase          5712
small_business          4064
car                     3149
medical                 2633
moving                  1675
vacation                1453
wedding                 1345
house                   1316
renewable_energy         206
educational              199
Name: Purpose_Of_Loan, dtype: int64

In [18]:
df['Length_Employed'].value_counts()

10+ years    88328
2 years      24420
3 years      21478
< 1 year     21133
5 years      18042
1 year       17336
4 years      16393
7 years      15444
6 years      15351
8 years      13115
9 years      10503
Name: Length_Employed, dtype: int64

In [19]:
df['Income_Verified'].value_counts()

VERIFIED - income           99076
VERIFIED - income source    88253
not verified                86521
Name: Income_Verified, dtype: int64

In [20]:
df['Home_Owner'].value_counts()

Mortgage    117270
Rent         93542
Own          20879
Other           79
None            20
Name: Home_Owner, dtype: int64

In [21]:
df['Gender'].value_counts()

Male      194993
Female     78857
Name: Gender, dtype: int64

# Below code generates new missing value Bool feature for each column , results with yes for msissing rows

In [22]:
cols_with_missing = [col for col in df.columns
                     if df[col].isnull().any()]
for col in cols_with_missing:
    df[col + '_was_missing'] = df[col].isnull()

# Interaction Features

In [23]:
interaction1=df['Income_Verified']+'_'+df['Home_Owner']
interaction2=df['Income_Verified']+'_'+df['Purpose_Of_Loan']
interaction3=df['Income_Verified']+'_'+df['Gender']

interaction4=df['Home_Owner']+'_'+df['Purpose_Of_Loan']
interaction5=df['Home_Owner']+'_'+df['Gender']

interaction6=df['Purpose_Of_Loan']+'_'+df['Gender']

df['T_C']=interaction1
df['T_D']=interaction2
df['T_G']=interaction3
df['C_D']=interaction4
df['C_G']=interaction5
df['D_G']=interaction6


# Label Encoding

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in ['Income_Verified','Home_Owner','Purpose_Of_Loan','Gender','T_C','T_D','T_G','C_D','C_G','D_G']:
    df[col]=  df[col].astype('str')
    df[col]= le.fit_transform(df[col]) 

# Frequency Encoding (didnt workout for other cat feature)

In [25]:
fe_pol = (df.groupby('Home_Owner').size()) / len(df)
df['Home_Owner'] = df['Home_Owner'].apply(lambda x : fe_pol[x])

# Mapping Values

In [26]:
df['Length_Employed'] = df['Length_Employed'].map({'< 1 year': 0,
                                                  '1 year': 1,
                                                  '2 years': 2,
                                                  '3 years': 3,
                                                  '4 years': 4,
                                                  '5 years': 5,
                                                  '6 years': 6,
                                                  '7 years': 7,
                                                  '8 years': 8,
                                                  '9 years': 9,
                                                  '10+ years': 10
                                                 })

In [27]:
df['Loan_Amount_Requested'] = df['Loan_Amount_Requested'].str.replace(',', '').astype(float)

In [28]:
df_all_corr = df.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
df_all_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
df_all_corr[df_all_corr['Feature 1'] == 'Annual_Income']

,Feature 1,Feature 2,Correlation Coefficient
24,Annual_Income,Annual_Income,1.000000
61,Annual_Income,Loan_Amount_Requested,0.349145
87,Annual_Income,Total_Accounts,0.211464
100,Annual_Income,Debt_To_Income,0.180756
117,Annual_Income,Number_Open_Accounts,0.149062
120,Annual_Income,C_G,0.140613
122,Annual_Income,T_C,0.139229
130,Annual_Income,C_D,0.131918
162,Annual_Income,Income_Verified,0.100244
176,Annual_Income,Length_Employed_was_missing,0.088556


In [29]:
df.reset_index(inplace=True)

# Filling Null values of Annual Income with median value of Loan amount since they had high correlation of 0.41. Remaining feature null values are handled by lgbm model

In [30]:
Anualincome_by_lA = df.groupby(['Loan_Amount_Requested']).median()['Annual_Income']
df['Annual_Income'] = df.groupby(['Loan_Amount_Requested'])['Annual_Income'].apply(lambda x: x.fillna(x.median()))

In [31]:
del df['index']

# Some Feature Engineering

In [32]:
df['Loan_Income_Ratio'] = df['Loan_Amount_Requested']/df['Annual_Income']

df['monthly_income'] = df['Annual_Income']/12

df['debt_monthly'] = (df['Debt_To_Income']*df['monthly_income'])/100

df['acc_ratio'] = df['Number_Open_Accounts']/df['Total_Accounts']

#df['term'] = df['Loan_Amount_Requested']/df['debt_monthly']

#df['debt'] = (df['Debt_To_Income']*df['monthly_income'])/100

#df['Saving'] = df['Annual_Income']-df['debt']


# Getting Back train test Data

In [33]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [34]:
train.columns

Index(['Annual_Income', 'Debt_To_Income', 'Gender', 'Home_Owner',
       'Income_Verified', 'Inquiries_Last_6Mo', 'Interest_Rate',
       'Length_Employed', 'Loan_Amount_Requested', 'Loan_ID',
       'Months_Since_Deliquency', 'Number_Open_Accounts', 'Purpose_Of_Loan',
       'Total_Accounts', 'Annual_Income_was_missing', 'Home_Owner_was_missing',
       'Interest_Rate_was_missing', 'Length_Employed_was_missing',
       'Months_Since_Deliquency_was_missing', 'T_C', 'T_D', 'T_G', 'C_D',
       'C_G', 'D_G', 'Loan_Income_Ratio', 'monthly_income', 'debt_monthly',
       'acc_ratio'],
      dtype='object')

In [35]:
# Find correlations with the target and sort
correlations = train.corr()['Interest_Rate'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 Number_Open_Accounts           0.002847
Length_Employed_was_missing    0.008337
Length_Employed                0.030404
C_G                            0.052393
debt_monthly                   0.053329
C_D                            0.060046
acc_ratio                      0.077913
D_G                            0.106949
Purpose_Of_Loan                0.107253
Loan_Amount_Requested          0.132486
Debt_To_Income                 0.152472
Inquiries_Last_6Mo             0.188987
Loan_Income_Ratio              0.197520
Interest_Rate                  1.000000
Interest_Rate_was_missing           NaN
Name: Interest_Rate, dtype: float64

Most Negative Correlations:
 Income_Verified                       -0.213371
T_G                                   -0.119025
Months_Since_Deliquency_was_missing   -0.113893
T_D                                   -0.098518
T_C                                   -0.084304
Annual_Income                         -0.046397
monthly_income   

In [36]:
x=train.drop(columns={'Loan_ID','Interest_Rate'},axis=1)
y=train.loc[:,['Interest_Rate']]
test=test.drop(columns={'Loan_ID','Interest_Rate'},axis=1)

# using Standart Scaler

In [37]:
st=StandardScaler()
st.fit(x)
x=st.transform(x)
test=st.transform(test)
x=pd.DataFrame(x)
test=pd.DataFrame(test)

In [38]:
# F1 weight metric faced some error while using this, had no enough time to correct it

In [39]:
def lgb_f1_score(truth, predictions):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='weighted')
    return ('weighted', f1, True) 


In [ ]:
err = [] 
y_pred_tot_lgm = []



fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
i = 1

for train_index, test_index in fold.split(x, y):
    x_train, x_val = x.iloc[train_index], x.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    m = LGBMClassifier(boosting_type='gbdt',
                       objective='multi:softmax',
                       num_class=3,
                       max_depth=5,
                       learning_rate=0.08,
                       n_estimators=10000,
                       min_child_weight=0.5,
                       colsample_bytree=0.9,
                       random_state=2020)
    m.fit(x_train, y_train,eval_set=[(x_train,y_train),(x_val, y_val)],#eval_metric=lgb_f1_score(pred_y,y_val),
          early_stopping_rounds=100,verbose=200)
    pred_y = m.predict(x_val)
    print(i, " err_lgm: ", f1_score(y_val,pred_y,average='weighted'))
    err.append(f1_score(y_val,pred_y,average='weighted'))
    pred_test = m.predict(test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)


C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ABetta\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training until validation scores don't improve for 100 rounds
[200]	training's multi_logloss: 0.891522	valid_1's multi_logloss: 0.917986
[400]	training's multi_logloss: 0.868247	valid_1's multi_logloss: 0.914812
Early stopping, best iteration is:
[471]	training's multi_logloss: 0.861464	valid_1's multi_logloss: 0.914729
1  err_lgm:  0.5403972287499895
Training until validation scores don't improve for 100 rounds
[200]	training's multi_logloss: 0.892485	valid_1's multi_logloss: 0.914162
[400]	training's multi_logloss: 0.868627	valid_1's multi_logloss: 0.911001
[600]	training's multi_logloss: 0.848949	valid_1's multi_logloss: 0.910558
Early stopping, best iteration is:
[534]	training's multi_logloss: 0.855115	valid_1's multi_logloss: 0.910421
2  err_lgm:  0.5408986391097383
Training until validation scores don't improve for 100 rounds
[200]	training's multi_logloss: 0.890965	valid_1's multi_logloss: 0.922406
[400]	training's multi_logloss: 0.867011	valid_1's multi_logloss: 0.920061
Early

In [ ]:
err

In [ ]:
np.sum(err)/5

In [ ]:
type(y_pred_tot_lgm)

In [ ]:
d = pd.DataFrame()
for i in range(5):
    d = pd.concat([d,pd.DataFrame(y_pred_tot_lgm[i])],axis=1)
d.columns=['1','2','3','4','5']

In [ ]:

re = d.mode(axis=1)[0]

In [ ]:
#pred=pd.DataFrame(y_pred_tot_lgm[0])
sub['Interest_Rate']=re
sub.to_csv('finalsub.csv',index=False)